In [26]:
  
import warnings

from allennlp import predictors
from allennlp.predictors import Predictor
from allennlp.models.archival import load_archive
import json
import docx2txt
import re

In [27]:
def writeToFile(fileName, content, mode):
    if len(content) == 0:
        return
    f = open(fileName, mode)
    f.write(content)
    f.close()


In [17]:
def generateThePredictor(preTrainedModelName):
  archive = load_archive(preTrainedModelName)
  return Predictor.from_archive(archive, "semantic-role-labeling")  

In [18]:
def generateTheSRLTag(sentence, predictor):
    return predictor.predict(sentence)

In [19]:
def preProcessTheTaggedFile(fileName):
    taggedContent = docx2txt.process(fileName)
    
    lines = taggedContent.split('\n')
    
    tactic = ''
    techniqueId = ''
    techniqueName = ''
    procedureFound = False
    procedureContent = ''
    sentenceListDict = {}
    sentenceList = []

    for line in lines:

        if len(line.strip()) == 0:
            continue
        elif '<tactic>' in line.lower():
            try:
                tactic = re.sub(r'<tactic>|<\\tactic>', '', line.lower())
            except:
                print(line)
                       
        elif '<techniquename>' in line.lower():
            try:
                techniqueName = re.sub(r'<techniquename>|<\\techniquename>', '', line.lower())
            except:
                print(line)
        elif '<id>' in line.lower():
            try:
                techniqueId = re.sub(r'<id>|<\\id>', '', line.lower())
            except:
                print(line, '\n')
        elif '<procedure>' in line.lower() or '<\\procedure>' in line.lower() or procedureFound:

            procedureFound = True
            procedureContent += re.sub(r'<([a-zA-Z\s]*)>|<\\([a-zA-Z\s]*)>', '', line.lower())
            if '<\\procedure>' in line.lower():
                
                sentenceList = procedureContent.split('.')
                key = techniqueName.strip() + '_' + techniqueId.strip()

                if key.lower() in sentenceListDict.keys():
                    sentenceListDict[key.lower()].append(sentenceList)
                else:
                    sentenceListDict[key.lower()] = [sentenceList]

                procedureFound = False
                procedureContent = ''
    return sentenceListDict

In [20]:
def parseTaggedSentence(taggedSRLSentence, fileName, mode):
    lineTowrite = 'Id, What, Where, How, How:What, How:Where, Why, Why:What, Why:Where, When, Miscelle\n'
    for tagged in taggedSRLSentence['verbs']:
        tagcount = sum((len(tag) > 1 or tag != "O") for tag in tagged['tags'])
        if tagcount > 2 :
            extractedTags = re.findall(r'\[.*?\]', tagged['description'])

            for t in extractedTags:
                if len(lineTowrite.strip()) == 0:
                    lineTowrite += re.sub(r'\[|\]', '', t)
                else:
                    lineTowrite +=  ', ' + re.sub(r'\[|\]', '', t)
            lineTowrite = "\"" + re.sub(r'\"|\'', '', tagged['description']) + "\"" + ', ' + lineTowrite + '\n'

            writeToFile(fileName, lineTowrite, mode)
            lineTowrite = ''

    

In [21]:
def writeAllSRLToSeparateFile(path, mode, sentencesByTechnique):
    line = 'Id, What, Where, How, How:What, How:Where, Why, Why: What, Why:Where, When, Miscelle\n'
    for key in sentencesByTechnique:
        k = re.sub(r'/','',key)
        lineTowrite = 'Id, What, Where, How, How:What, How:Where, Why, Why: What, Why:Where, When, Miscelle\n'
        
        for procedure in sentencesByTechnique[key]:
            for sentence in procedure:
                if len(sentence.strip()) == 0:
                    continue
                lablledSentence = generateTheSRLTag(sentence, predictor)
                
                #parseTaggedSentence(lablledSentence, path + k + '.csv', mode)
                l = parseTaggedSentenceWh(lablledSentence, k, path + k + '.csv', mode)
                lineTowrite += l
                line += l
        writeToFile(path + k + '.csv', lineTowrite, mode)
        
    writeToFile(path + 'AllTechniques.csv', line, mode)

In [22]:

def parseTaggedSentenceWh(taggedSRLSentence, k, fileName, mode):
    
    ids = k.split('_')
    
    lineTowrite = ""

    tagList = dict()
    for tagged in taggedSRLSentence['verbs']:
        
        tagcount = sum((len(tag) > 1 or tag != "O") for tag in tagged['tags'])
        if tagcount > 2 :
            extractedTags = re.findall(r'\[.*?\]', tagged['description'])

            for t in extractedTags:
                if 'MOD:' in t or 'ARG0:' in t:
                    continue
                elif 'LOC:' in t:
                    tmp = re.sub(r'\[|\]', '', t).split(":")
                    if 'misc' in tagList.keys():
                        tagList['misc'] = tagList['misc']  + ';' + tmp[1].strip()
                    else:
                        tagList['misc'] = tmp[1].strip()
                elif 'ADV:' in t or "TMP:" in t:
                    tmp = re.sub(r'\[|\]', '', t).split(":")
                    if 'when' in tagList.keys():
                        tagList['when'] = tagList['when']  + ';' + tmp[1].strip()
                    else:
                        tagList['when'] = tmp[1].strip()        
                elif 'V:' in t:
                    
                    tmp = re.sub(r'\[|\]', '', t).split(":")

                    if 'what' in tagList.keys():
                        if 'how' in tagList.keys() and tmp[1] in tagList['how']:
                            if 'how:what' in tagList.keys():
                                tagList['how:what'] = tagList['how:what'] + ";" + tmp[1].strip()
                            else:
                                tagList['how:what'] = tmp[1].strip()
                        elif 'why' in tagList.keys() and tmp[1] in tagList['why']:
                            if 'why:what' in tagList.keys():
                                tagList['why:what'] = tagList['why:what'] + ";" + tmp[1].strip()
                            else:
                                tagList['why:what'] = tmp[1].strip()
                           
                        else:
                            if 'where' in tagList.keys() :
                                lineTowrite += prepareDataToWrite(ids[1], tagList)
                                tagList.clear()
                                tagList['what'] = tmp[1].strip()
                            else:
                                tagList['what'] = tagList['what'] + ";" + tmp[1].strip()
                            
                            
                    else:
                        tagList['what'] = tmp[1].strip()
                    
                elif 'ARG1:' in t:
                    
                    tmp = re.sub(r'\[|\]', '', t).split(":")
                    if 'where' in tagList.keys():
                        if 'how' in tagList.keys() and tmp[1] in tagList['how']:
                            if 'how:where' in tagList.keys():
                                tagList['how:where'] = tagList['how:where'] + ";" + tmp[1].strip()
                            else:
                                tagList['how:where'] = tmp[1].strip()
                        elif 'why' in tagList.keys() and tmp[1] in tagList['why']:
                            if 'why:where' in tagList.keys():
                                tagList['why:where'] = tagList['why:where'] + ";" + tmp[1].strip()
                            else:
                                tagList['why:where'] = tmp[1].strip()
                                
                        else:
                            if 'what' in tagList.keys() :
                                lineTowrite += prepareDataToWrite(ids[1], tagList)
                                tagList.clear()
                                tagList['where'] = tmp[1].strip()
                            else:
                                tagList['where'] = tagList['where'] + ';' + tmp[1].strip() 
                        
                    
                    else:
                        tagList['where'] = tmp[1].strip()
                elif 'PNC' in t or 'CAU' in t or 'ARG2:' in t or 'PRP:' in t:
                    tmp = re.sub(r'\[|\]', '', t).split(":")
                    if 'why' in tagList.keys():
                        tagList['why'] = tagList['why'] + ";" +  tmp[1].strip()
                    else:
                        tagList['why'] = tmp[1].strip()
                    
                elif 'MNR:' in t or 'DIR' in t:
                    tmp = re.sub(r'\[|\]', '', t).split(":")
                    if 'how' in tagList.keys():
                        tagList['how'] = tagList['how'] + ";" +  tmp[1].strip()
                    else:
                        tagList['how'] = tmp[1].strip()
    
    lineTowrite += prepareDataToWrite(ids[1], tagList)
    tagList.clear()
    return lineTowrite

In [23]:
def prepareDataToWrite(id, tagList):
    if len(tagList) == 0: 
        return ''
    #print(tagList)
    lineTowrite = id
    if 'what' in tagList.keys():
        lineTowrite += ",\"" + tagList['what'] + "\""
    else:
        lineTowrite += " ,"
        
    if 'where' in tagList.keys():
        lineTowrite += ",\"" + tagList['where'] + "\""
    else:
        lineTowrite += " ,"
        
    if 'how' in tagList.keys():
        lineTowrite += ",\"" + tagList['how'] + "\""
    else:
        lineTowrite += " ,"
        
    if 'how:what' in tagList.keys():
        lineTowrite += ",\"" + tagList['how:what'] + "\""
    else:
        lineTowrite += " ,"
        
    if 'how:where' in tagList.keys():
        lineTowrite += ",\"" + tagList['how:where'] + "\""
    else:
        lineTowrite += ' , '
        
    if 'why' in tagList.keys():
        lineTowrite += ",\"" + tagList['why'] + "\""
    else:
        lineTowrite += " ,"
        
    if 'why:what' in tagList.keys():
        lineTowrite += ",\"" + tagList['why:what'] + "\""
    else:
        lineTowrite += " ,"
        
    if 'why:where' in tagList.keys():
        lineTowrite += ",\"" + tagList['why:where'] + "\""
    else:
        lineTowrite += " ,"
        
    if 'when' in tagList.keys():
        lineTowrite += ",\"" + tagList['when'] + "\""
    else:
        lineTowrite += " ,"
        
    if 'misc' in tagList.keys():
        lineTowrite += ",\"" + tagList['misc'] + "\""
    else:
        lineTowrite += " ,"
    lineTowrite += '\n'
    #print(lineTowrite)
    return lineTowrite


In [10]:

            
predictor = generateThePredictor("bert-base-srl-2019.06.17.tar.gz")
        

In [24]:
sentencesByTechnique1 = preProcessTheTaggedFile("TaggedTechniques_Mohi.docx")#('tacticTest.docx')#
writeAllSRLToSeparateFile('SRLTaggedV1/', 'a', sentencesByTechnique1)

In [25]:
sentencesByTechnique1 = preProcessTheTaggedFile("TaggedTechniques_Ruhani_Revised.docx")
writeAllSRLToSeparateFile('SRLTaggedV1/Ru_', 'a', sentencesByTechnique1)

In [16]:
sentencesByTechnique3 = preProcessTheTaggedFile("tactictest1.docx")
predictor = generateThePredictor("bert-base-srl-2019.06.17.tar.gz")


In [86]:
writeAllSRLToSeparateFile('SRLTagged/', 'a', sentencesByTechnique3)

t1171,"spoof","an authoritative source for name resolution","by responding to llmnr ( udp 5355)/nbt - ns ( udp 137 ) traffic as if they know the identity of the requested host;the adversary controlled system;as if they know the identity of the requested host;effectively","responding;know","to llmnr ( udp 5355)/nbt - ns ( udp 137 ) traffic;the identity of the requested host" , , ,,"effectively poisoning the service so that the victims will communicate with;effectively poisoning the service so that the victims will communicate with the adversary","on a victim network"

t1171,"poisoning","the service" , , , ,"so that the victims will communicate with the adversary controlled system;with the adversary controlled system","communicate;controlled" , , ,

t1171,"belongs","to a resource that requires identification / authentication" , , ,  , , , , ,

t1171,"requires","identification / authentication" , , ,  , , ,,"if the requested host belongs to a resource that requires identification / authen